In [135]:
# Import Required Libraries
import pandas as pd
import numpy as np
from faker import Faker

fake = Faker()

In [136]:
# Create the Billing Table
FactTable = pd.read_csv("data/FactTable.csv", parse_dates=['dimDateServicePK'], dayfirst=True)
DimHospital = pd.read_csv("data/DimHospital.csv")
Billing = pd.merge(FactTable, DimHospital, on='dimLocationPK', how='left')

# Apply Transformations
Billing["PaymentRecieved"] = Billing["Patient_Payment"] + Billing["AR"]
Billing["DiagnosisReportRef"] = "DIAG-" + Billing["FactTablePK"].astype(str)
Billing["ServiceReportRef"] = "SERV-" + Billing["FactTablePK"].astype(str)

column_mapping = {
    "FactTablePK" : "BillNbr",
    "dimDatePostPK" : "BillDate",
    "dimPatientPK" : "PatientNbr",
    "dimPhysicianPK" : "ProviderNbr",
    "LocationName" : "Facility",
    "Gross Expenses" : "BillAmount",
    "Insurance_Payment" : "InsuranceCoverage"
}

# Rename Columns
Billing = Billing.rename(columns=column_mapping)

# Select Columns
Billing = Billing[["BillNbr", "BillDate", "PatientNbr", "ProviderNbr", "Facility", "DiagnosisReportRef", "ServiceReportRef", "BillAmount", "InsuranceCoverage", "PaymentRecieved"]]

# Write the new Data to CSV File
Billing.to_csv('Spaghetti/Billing.csv', index=False)

C:\Users\murta\AppData\Local\Temp\ipykernel_2256\536781932.py:2: DtypeWarning: Columns (14) have mixed types. Specify dtype option on import or set low_memory=False.
  FactTable = pd.read_csv("data/FactTable.csv", parse_dates=['dimDateServicePK'], dayfirst=True)


In [137]:
# Create the Patient Table
DimPatient = pd.read_csv("data/DimPatient.csv")
MinServiceDate = FactTable.groupby("dimPatientPK").agg({"dimDateServicePK": "min"}).reset_index()
Patient = pd.merge(DimPatient, MinServiceDate, on='dimPatientPK', how='left')

# Apply Transformations
Patient.rename(columns={"Year of Birth": "year", "Month": "month", "Day": "day"}, inplace=True)
Patient["DOB"] = pd.to_datetime(Patient[["year", "month", "day"]])
Patient["RegistrationDate"] = Patient["dimDateServicePK"] - pd.to_timedelta(np.random.randint(1, 10, size=len(Patient)), unit='days')
Patient[["Address1", "Address2"]] = Patient.apply(lambda _: pd.Series([fake.secondary_address(), fake.street_address()]), axis=1)

column_mapping = {
    "dimPatientPK" : "PatientNbr",
    "PatientNumber" : "HealthCardNbr",
    "PatientGender" : "Gender",
    "Zip Codes" : "Zip"
}

# Rename Columns
Patient = Patient.rename(columns=column_mapping)

# Select Columns
Patient = Patient[["PatientNbr", "RegistrationDate", "HealthCardNbr", "FirstName", "LastName", "Email", "Gender", "DOB", "Address1", "Address2", "City", "State", "Zip"]]

# Write the new Data to CSV File
Patient.to_csv('Spaghetti/Patient.csv', index=False)

,PatientNbr,RegistrationDate,HealthCardNbr,FirstName,LastName,Email,Gender,DOB,Address1,Address2,City,State,Zip
0,4760825,2019-12-11,21376821,Melanie,Greene,melanie.greene@datacourse.com,Female,1925-07-31,Apt. 257,503 Barnes Run Suite 267,Auburn,Alabama,36849
1,4975268,2019-12-09,21391171,Neil,Pullman,neil.pullman@datacourse.com,Male,1914-05-11,Suite 562,842 Madison Glen,Auburn,Alabama,36849
2,5331866,2020-01-30,21385004,Kimberly,Mills,kimberly.mills@datacourse.com,Female,1935-10-12,Apt. 871,97476 Johnson Walk Suite 651,Auburn,Alabama,36849
3,5431278,2019-12-15,21365432,Dan,Wilkins,dan.wilkins@datacourse.com,Male,1981-06-19,Apt. 015,794 Brandon Shoals,Auburn,Alabama,36849
4,5498935,2019-12-08,21387538,Benjamin,Cornish,benjamin.cornish@datacourse.com,Male,1972-06-16,Suite 896,498 Robert Mountain Apt. 124,Auburn,Alabama,36849
...,...,...,...,...,...,...,...,...,...,...,...,...,...
5112,5460538,2019-12-12,21376443,Edward,Paterson,edward.paterson@datacourse.com,Male,1996-07-15,Apt. 452,50746 Torres Parks Suite 634,Cheyenne,Wyoming,82009
5113,5501187,2019-12-14,21377493,Julian,Fisher,julian.fisher@datacourse.com,Male,1991-03-09,Suite 885,274 Bryan Flat,Cheyenne,Wyoming,82009
5114,5506864,2019-12-16,21377661,Madeleine,Alsop,madeleine.alsop@datacourse.com,Female,1985-05-29,Suite 518,64896 Julie Groves,Cheyenne,Wyoming,82009
5115,5517589,2019-12-08,21372453,Alison,Coleman,alison.coleman@datacourse.com,Female,1937-05-23,Apt. 866,95562 Beard Corners,Cheyenne,Wyoming,82009


In [138]:
# FactTable["dimDateServicePK"] = pd.to_datetime(FactTable["dimDateServicePK"], errors='coerce', dayfirst=True)
# FactTable["dimDateServicePK"] = pd.to_datetime(FactTable["dimDateServicePK"], errors='coerce', dayfirst=True)
Patient#.info()

,PatientNbr,RegistrationDate,HealthCardNbr,FirstName,LastName,Email,Gender,DOB,Address1,Address2,City,State,Zip
0,4760825,2019-12-11,21376821,Melanie,Greene,melanie.greene@datacourse.com,Female,1925-07-31,Apt. 257,503 Barnes Run Suite 267,Auburn,Alabama,36849
1,4975268,2019-12-09,21391171,Neil,Pullman,neil.pullman@datacourse.com,Male,1914-05-11,Suite 562,842 Madison Glen,Auburn,Alabama,36849
2,5331866,2020-01-30,21385004,Kimberly,Mills,kimberly.mills@datacourse.com,Female,1935-10-12,Apt. 871,97476 Johnson Walk Suite 651,Auburn,Alabama,36849
3,5431278,2019-12-15,21365432,Dan,Wilkins,dan.wilkins@datacourse.com,Male,1981-06-19,Apt. 015,794 Brandon Shoals,Auburn,Alabama,36849
4,5498935,2019-12-08,21387538,Benjamin,Cornish,benjamin.cornish@datacourse.com,Male,1972-06-16,Suite 896,498 Robert Mountain Apt. 124,Auburn,Alabama,36849
...,...,...,...,...,...,...,...,...,...,...,...,...,...
5112,5460538,2019-12-12,21376443,Edward,Paterson,edward.paterson@datacourse.com,Male,1996-07-15,Apt. 452,50746 Torres Parks Suite 634,Cheyenne,Wyoming,82009
5113,5501187,2019-12-14,21377493,Julian,Fisher,julian.fisher@datacourse.com,Male,1991-03-09,Suite 885,274 Bryan Flat,Cheyenne,Wyoming,82009
5114,5506864,2019-12-16,21377661,Madeleine,Alsop,madeleine.alsop@datacourse.com,Female,1985-05-29,Suite 518,64896 Julie Groves,Cheyenne,Wyoming,82009
5115,5517589,2019-12-08,21372453,Alison,Coleman,alison.coleman@datacourse.com,Female,1937-05-23,Apt. 866,95562 Beard Corners,Cheyenne,Wyoming,82009
